<a href="https://colab.research.google.com/github/yura-eng/HUFSworkshop/blob/main/gradio_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python to app: Gradio
##[Gradio webpage](https://www.gradio.app/)

##1.  간단한 더하기 계산기 app

In [ ]:
def sum(a, b):

  c = int(a + b)
  c = '정답은 ' + str(c) + '입니다'
  return c

In [ ]:
sum(2,3)

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
iface = gr.Interface(sum, inputs = ["number", "number"], outputs = ["text"])
iface.launch()

## 2. Wordcloud app

### Sample text
- At the heart of Trump’s designs is a plan to concentrate the powers of the federal government inside the office of the presidency. His most ambitious proposals, he says, would be implemented through executive action: shutting down the southern border and initiating mass deportations of migrants; imposing across-the-board tariffs on imported goods; and withdrawing the U.S. from its role as a bulwark against tyranny abroad. He would attempt to castrate the federal bureaucracy by firing civil servants at will, and he would aim to neutralize Capitol Hill’s power of the purse by controlling congressionally-appropriated funds. To further his vision of enhanced executive authority, Trump would staff his Administration with acolytes committed to removing the guardrails that persisted in his first term. “The President never had a policy process that was designed to give him what he actually wanted,” says Russ Vought, Trump’s former Office of Management and Budget director who now heads a think tank preparing for a second Trump term. “You had various institutional obstacles.”

In [ ]:
# Step 1: Install required libraries in Colab
!pip install gradio wordcloud matplotlib

# Step 2: Import the necessary libraries
import gradio as gr
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import numpy as np
from io import BytesIO
from PIL import Image

# Step 3: Define a function to create the word cloud
def generate_wordcloud(text):
    # Basic text cleaning: remove special characters and limit text size
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    if len(text) > 5000:  # Limit the length of the text to prevent errors
        text = text[:5000]

    try:
        # Create a WordCloud object
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

        # Convert the WordCloud to an image format that Gradio can display directly
        image = wordcloud.to_image()
        return image
    except Exception as e:
        print(f"Error generating word cloud: {e}")
        return None

# Step 4: Set up the Gradio interface
interface = gr.Interface(fn=generate_wordcloud,
                         inputs="text",
                         outputs="image",
                         title="Word Cloud Generator",
                         description="Enter some text to generate a word cloud of the most common words.")

# Step 5: Launch the app
interface.launch()


#3. Sentence-by-sentence Read -Aloud practice

###Sample text

In the small mountain village of Echo Ridge, adventure was a part of everyday life. Nestled among towering peaks, the village was said to be protected by the "Guardian of the Glen," a massive eagle that supposedly watched over the villagers from its perch high in the mountains. The legend inspired many adventurous tales among the villagers, especially the children.

###Version 1. 입력부분 어색

In [ ]:
!pip install gradio gtts nltk pypng

In [ ]:
#@markdown Gradio app for Read-Aloud practice (by sentences)
import gradio as gr
from gtts import gTTS
from nltk import tokenize
import os

# Import necessary nltk libraries
import nltk
nltk.download('punkt')

mytext = input()

# Tokenize the predefined text into sentences
sentences = tokenize.sent_tokenize(mytext)

def generate_audio(selected_item):
    index_str = selected_item.split('.')[0]
    index = int(index_str) - 1  # Adjust for 0-based index

    if 0 <= index < len(sentences):
        sentence = sentences[index]
        tts = gTTS(text=sentence, lang='en')
        audio_path = f'sentence_{index + 1}.mp3'
        tts.save(audio_path)
        return audio_path
    else:
        return "Invalid sentence number"

# Gradio interface
iface = gr.Interface(
    fn=generate_audio,
    inputs=gr.Dropdown(choices=[f"{i + 1}. {s}" for i, s in enumerate(sentences)], label="Select Sentence"),
    outputs=gr.Audio(label="Audio of Selected Sentence"),
    title="Text to Audio by Sentence",
    description="Select a sentence number to hear it spoken."
)

iface.launch()


###Version 2. 입력부분 개선

In [ ]:
#@markdown Split sentences
import gradio as gr
from gtts import gTTS
from nltk import tokenize
import os

# Import necessary nltk libraries
import nltk
nltk.download('punkt')

# Global variable to store sentences
sentences = []

# Function to process text and generate sentence options
def process_text(mytext):
    global sentences
    sentences = tokenize.sent_tokenize(mytext)
    return [f"{i + 1}. {s}" for i, s in enumerate(sentences)]

# Function to generate audio for the selected sentence
def generate_audio(selected_item):
    if not selected_item:
        return None

    index = int(selected_item.split('.')[0]) - 1  # Adjust for 0-based index

    if 0 <= index < len(sentences):
        sentence = sentences[index]
        tts = gTTS(text=sentence, lang='en')
        audio_path = f'sentence_{index + 1}.mp3'
        tts.save(audio_path)
        return audio_path
    else:
        return None

# Function to update dropdown choices based on text input
def update_dropdown(mytext):
    choices = process_text(mytext)
    return gr.update(choices=choices)

# Create a Gradio Blocks app
with gr.Blocks() as app:
    with gr.Row():
        textbox = gr.Textbox(label="Enter your text here")
    with gr.Row():
        submit_button = gr.Button("Submit")
    with gr.Row():
        dropdown = gr.Dropdown(choices=[], label="Select Sentence")
    with gr.Row():
        audio_output = gr.Audio(label="Audio of Selected Sentence")

    submit_button.click(fn=update_dropdown, inputs=textbox, outputs=dropdown)
    dropdown.change(fn=generate_audio, inputs=dropdown, outputs=audio_output)

app.launch()


###3. 입력부분 개선 - 전체문장 or 문장별

In [ ]:
import gradio as gr
from gtts import gTTS
from nltk import tokenize
import os

# Import necessary nltk libraries
import nltk
nltk.download('punkt')

# Global variable to store sentences and the entire text
sentences = []
full_text = ""

# Function to process text and generate sentence options
def process_text(mytext):
    global sentences, full_text
    full_text = mytext
    sentences = tokenize.sent_tokenize(mytext)
    choices = ["Play the whole text"] + [f"{i + 1}. {s}" for i, s in enumerate(sentences)]
    return choices

# Function to generate audio for the selected item
def generate_audio(selected_item):
    global full_text

    if not selected_item:
        return None

    if selected_item == "Play the whole text":
        tts = gTTS(text=full_text, lang='en')
        audio_path = 'full_text.mp3'
        tts.save(audio_path)
        return audio_path

    index = int(selected_item.split('.')[0]) - 1  # Adjust for 0-based index

    if 0 <= index < len(sentences):
        sentence = sentences[index]
        tts = gTTS(text=sentence, lang='en')
        audio_path = f'sentence_{index + 1}.mp3'
        tts.save(audio_path)
        return audio_path
    else:
        return None

# Function to update dropdown choices based on text input
def update_dropdown(mytext):
    choices = process_text(mytext)
    return gr.update(choices=choices)

# Create a Gradio Blocks app
with gr.Blocks() as app:
    with gr.Row():
        textbox = gr.Textbox(label="Enter your text here")
    with gr.Row():
        submit_button = gr.Button("Submit")
    with gr.Row():
        dropdown = gr.Dropdown(choices=[], label="Select Sentence")
    with gr.Row():
        audio_output = gr.Audio(label="Audio of Selected Sentence")

    submit_button.click(fn=update_dropdown, inputs=textbox, outputs=dropdown)
    dropdown.change(fn=generate_audio, inputs=dropdown, outputs=audio_output)

app.launch()